# Actor Critic for Cartpole

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.distributions import Categorical

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import gym

### Cartpole environment

In [2]:
env = gym.make('CartPole-v1')

### Actor Critic agent

Common model for actor and critic

In [ ]:
class ActorCritic(nn.Module):
    
    def __init__(self):
        super(ActorCritic, self).__init__()
        
        # Common network
        self.fc1 = nn.Linear(in_features=4, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=2)
        
        # Actor head
        self.actor_head = nn.Linear(in_features=2, out_features=2)
        
        # Critic head
        self.critic_head = nn.Linear(in_features=2, out_features=2)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        policy = F.softmax(self.actor_head(x))
        value = self.critic_head(x)
        return policy, value

class AC_Agent():
    
    def __init__(self, env, lr=0.001, dr=0.95):
        # Cartpole environment
        self.cartpole_env = env
        
        # Learning rates
        self.lr = lr
        # Discount rate
        self.dr = dr
        
        # Actor Critic model
        self.model = ActorCritic()
        self.optim = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        
        # Saving training curves
        self.rewards = []
        self.value_losses = []
        
    def choose_action(self, state):
        # Translate state to fit model input
        state = torch.from_numpy(state).float().unsqueeze(0)
        # Get actions probability distribution
        probs, _ = self.model(state)
        distrib = Categorical(probs)
        # Sample action
        action = distrib.sample()
        return action.item(), distrib.log_prob(action)
    
    def compute_loss(self, state, action, log_prob, reward, next_state, done, G=None):
        # Get value of state-action pair
        state = torch.from_numpy(state).float().unsqueeze(0)
        _, q_values = self.model(state)
        q_value = q_values[0, action]
        # Compute Actor's Loss
        L_actor = -log_prob * G
        if not done:
            if G is None:
                # Sample next action
                next_action, _ = self.choose_action(next_state)
                # Get value of next_state-next_action pair
                next_state = torch.from_numpy(next_state).float().unsqueeze(0)
                _, next_q_values = self.model(next_state)
                next_q_value = next_q_values[0, action]
                # Compute q_target
                q_target = torch.Tensor([reward]) + self.dr * next_q_value
            else:
                q_target = G
        else:
            q_target = torch.Tensor([reward])
        # Compute Critic's loss
        L_critic = (q_target - q_value) ** 2
        self.value_losses.append(L_critic)
        # Return complete loss
        return L_actor + L_critic
        
    def train(self, nb_episodes=1000, complete_return=False):
        for i in range(nb_episodes):
            state = env.reset()
            experiences = []
            for t in range(500):
                env.render()
                # Actor chooses action
                action, log_prob = self.choose_action(state)
                # Perform action
                next_state, reward, done, info = env.step(action)
                
                experiences.append((state, action, log_prob, reward, next_state, done))
                
                # Check for end state
                if done:
                    print("Episode #{} finished after {} timesteps.".format(i + 1, t + 1))
                    self.rewards.append(t + 1)
                    break
                
                state = next_state
                
            # Compute Loss
            loss = []
            for t in range(len(experiences)):
                if complete_return:
                    # Compute return
                    G = 0.0
                    for k in range(t, len(experiences)):
                        G += experiences[k][3] * self.dr ** k
                else:
                    G = None
                state, action, log_prob, reward, next_state, done = experiences[t]
                loss.append(self.compute_loss(state, action, log_prob, reward, next_state, done, G))
                
            # Learning step
            self.optim.zero_grad()
            loss = torch.cat(loss).sum()
            loss.backward()
            self.optim.step()
                
        self.cartpole_env.close()

In [ ]:
ac_agent = AC_Agent(env)
ac_agent.train(nb_episodes=10000, complete_return=True)

In [3]:
def compute_mov_avg(data: list, window=10):
    ids = [window * x for x in range(0, int(len(data) / window) + 1)]
    if ids[-1] < len(data) - 1:
        ids.append(len(data) - 1)
    mov_avg = []
    for i in range(1, len(ids)):
        mov_avg.append(sum(data[ids[i - 1]:ids[i]]) / len(data[ids[i - 1]:ids[i]]))
    return ids[1:], mov_avg

def display_metrics(data: list, legend: str, mov_avg=50):
    plt.figure(figsize=(15,5))
    # Plot data
    plt.plot(np.arange(len(data)), data, label=legend)
    # Plot moving average
    if len(data) > 5 * mov_avg:
        ids, avg = compute_mov_avg(data, mov_avg)
        plt.plot(ids, avg, label=legend + ' average')
    plt.legend()
    plt.show()

In [ ]:
display_metrics(ac_agent.rewards, "Reward")
display_metrics(ac_agent.value_losses, "Value Loss")

### Advantage Actor Critic agent

In [9]:
class AdvantageActorCritic(nn.Module):
    
    def __init__(self):
        super(AdvantageActorCritic, self).__init__()
        
        # Common network
        self.fc1 = nn.Linear(in_features=4, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=2)
        
        # Actor head
        self.actor_head = nn.Linear(in_features=2, out_features=2)
        
        # Critic head
        self.critic_head = nn.Linear(in_features=2, out_features=1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        
        policy = F.softmax(self.actor_head(x))
        value = self.critic_head(x)
        return policy, value

class A2C_Agent():
    
    def __init__(self, env, lr=0.001, dr=0.95):
        # Cartpole environment
        self.cartpole_env = env
        
        # Learning rates
        self.lr = lr
        # Discount rate
        self.dr = dr
        
        # Actor Critic model
        self.model = AdvantageActorCritic()
        self.optim = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        
        # Saving training curves
        self.rewards = []
        self.value_losses = []
        
    def choose_action(self, state):
        # Translate state to fit model input
        state = torch.from_numpy(state).float().unsqueeze(0)
        # Get actions probability distribution
        probs, _ = self.model(state)
        distrib = Categorical(probs)
        # Sample action
        action = distrib.sample()
        return action.item(), distrib.log_prob(action)
    
    def compute_loss(self, state, action, log_prob, reward, next_state, done, G=None):
        # Get value of state
        state = torch.from_numpy(state).float().unsqueeze(0)
        _, value = self.model(state)
        # Compute Q-target
        if not done:
            if G is None:
                # Get value of next_state
                next_state = torch.from_numpy(next_state).float().unsqueeze(0)
                _, next_value = self.model(next_state)
                # Compute q_target
                q_target = torch.Tensor([reward]) + self.dr * next_value
            else:
                q_target = G
        else:
            q_target = torch.Tensor([reward])
        # Compute Advantage
        advantage = q_target - value
        # Compute Actor's Loss
        L_actor = -log_prob * advantage
        # Compute Critic's loss
        L_critic = (q_target - value) ** 2
        self.value_losses.append(L_critic)
        # Return complete loss
        return L_actor + L_critic
        
    def train(self, nb_episodes=1000, complete_return=False):
        for i in range(nb_episodes):
            state = env.reset()
            experiences = []
            for t in range(500):
                env.render()
                # Actor chooses action
                action, log_prob = self.choose_action(state)
                # Perform action
                next_state, reward, done, info = env.step(action)
                
                experiences.append((state, action, log_prob, reward, next_state, done))
                
                # Check for end state
                if done:
                    print("Episode #{} finished after {} timesteps.".format(i + 1, t + 1))
                    self.rewards.append(t + 1)
                    break
                
                state = next_state
                
            # Compute Loss
            loss = []
            for t in range(len(experiences)):
                if complete_return:
                    # Compute return
                    G = 0.0
                    for k in range(t, len(experiences)):
                        G += experiences[k][3] * self.dr ** k
                else:
                    G = None
                state, action, log_prob, reward, next_state, done = experiences[t]
                loss.append(self.compute_loss(state, action, log_prob, reward, next_state, done, G))
                
            # Learning step
            self.optim.zero_grad()
            loss = torch.cat(loss).sum()
            loss.backward()
            self.optim.step()
                
        self.cartpole_env.close()

In [ ]:
a2c_agent = A2C_Agent(env)
a2c_agent.train(nb_episodes=15000, complete_return=True)

<ipython-input-9-68b65f219c42>:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  policy = F.softmax(self.actor_head(x))


Episode #1 finished after 10 timesteps.
Episode #2 finished after 10 timesteps.
Episode #3 finished after 12 timesteps.
Episode #4 finished after 14 timesteps.
Episode #5 finished after 12 timesteps.
Episode #6 finished after 9 timesteps.
Episode #7 finished after 11 timesteps.
Episode #8 finished after 15 timesteps.
Episode #9 finished after 51 timesteps.
Episode #10 finished after 12 timesteps.
Episode #11 finished after 21 timesteps.
Episode #12 finished after 18 timesteps.
Episode #13 finished after 17 timesteps.
Episode #14 finished after 19 timesteps.
Episode #15 finished after 10 timesteps.
Episode #16 finished after 12 timesteps.
Episode #17 finished after 24 timesteps.
Episode #18 finished after 15 timesteps.
Episode #19 finished after 12 timesteps.
Episode #20 finished after 17 timesteps.
Episode #21 finished after 26 timesteps.
Episode #22 finished after 9 timesteps.
Episode #23 finished after 11 timesteps.
Episode #24 finished after 16 timesteps.
Episode #25 finished after 

Episode #199 finished after 20 timesteps.
Episode #200 finished after 52 timesteps.
Episode #201 finished after 31 timesteps.
Episode #202 finished after 23 timesteps.
Episode #203 finished after 15 timesteps.
Episode #204 finished after 11 timesteps.
Episode #205 finished after 9 timesteps.
Episode #206 finished after 18 timesteps.
Episode #207 finished after 16 timesteps.
Episode #208 finished after 13 timesteps.
Episode #209 finished after 11 timesteps.
Episode #210 finished after 10 timesteps.
Episode #211 finished after 17 timesteps.
Episode #212 finished after 20 timesteps.
Episode #213 finished after 26 timesteps.
Episode #214 finished after 29 timesteps.
Episode #215 finished after 49 timesteps.
Episode #216 finished after 19 timesteps.
Episode #217 finished after 15 timesteps.
Episode #218 finished after 21 timesteps.
Episode #219 finished after 12 timesteps.
Episode #220 finished after 13 timesteps.
Episode #221 finished after 11 timesteps.
Episode #222 finished after 22 time

Episode #395 finished after 37 timesteps.
Episode #396 finished after 23 timesteps.
Episode #397 finished after 15 timesteps.
Episode #398 finished after 28 timesteps.
Episode #399 finished after 16 timesteps.
Episode #400 finished after 32 timesteps.
Episode #401 finished after 21 timesteps.
Episode #402 finished after 26 timesteps.
Episode #403 finished after 13 timesteps.
Episode #404 finished after 11 timesteps.
Episode #405 finished after 14 timesteps.
Episode #406 finished after 22 timesteps.
Episode #407 finished after 15 timesteps.
Episode #408 finished after 16 timesteps.
Episode #409 finished after 26 timesteps.
Episode #410 finished after 33 timesteps.
Episode #411 finished after 12 timesteps.
Episode #412 finished after 12 timesteps.
Episode #413 finished after 14 timesteps.
Episode #414 finished after 42 timesteps.
Episode #415 finished after 14 timesteps.
Episode #416 finished after 27 timesteps.
Episode #417 finished after 40 timesteps.
Episode #418 finished after 33 tim

Episode #591 finished after 28 timesteps.
Episode #592 finished after 26 timesteps.
Episode #593 finished after 18 timesteps.
Episode #594 finished after 20 timesteps.
Episode #595 finished after 25 timesteps.
Episode #596 finished after 35 timesteps.
Episode #597 finished after 13 timesteps.
Episode #598 finished after 35 timesteps.
Episode #599 finished after 19 timesteps.
Episode #600 finished after 10 timesteps.
Episode #601 finished after 67 timesteps.
Episode #602 finished after 33 timesteps.
Episode #603 finished after 33 timesteps.
Episode #604 finished after 50 timesteps.
Episode #605 finished after 12 timesteps.
Episode #606 finished after 21 timesteps.
Episode #607 finished after 26 timesteps.
Episode #608 finished after 13 timesteps.
Episode #609 finished after 16 timesteps.
Episode #610 finished after 39 timesteps.
Episode #611 finished after 19 timesteps.
Episode #612 finished after 18 timesteps.
Episode #613 finished after 29 timesteps.
Episode #614 finished after 15 tim

...

Episode #6510 finished after 93 timesteps.
Episode #6511 finished after 37 timesteps.
Episode #6512 finished after 113 timesteps.
Episode #6513 finished after 35 timesteps.
Episode #6514 finished after 164 timesteps.
Episode #6515 finished after 17 timesteps.
Episode #6516 finished after 66 timesteps.
Episode #6517 finished after 37 timesteps.
Episode #6518 finished after 112 timesteps.
Episode #6519 finished after 64 timesteps.
Episode #6520 finished after 71 timesteps.
Episode #6521 finished after 180 timesteps.
Episode #6522 finished after 52 timesteps.
Episode #6523 finished after 31 timesteps.
Episode #6524 finished after 43 timesteps.
Episode #6525 finished after 67 timesteps.
Episode #6526 finished after 65 timesteps.
Episode #6527 finished after 77 timesteps.
Episode #6528 finished after 99 timesteps.
Episode #6529 finished after 115 timesteps.
Episode #6530 finished after 107 timesteps.
Episode #6531 finished after 86 timesteps.
Episode #6532 finished after 66 timesteps.
Episo

Episode #6700 finished after 41 timesteps.
Episode #6701 finished after 129 timesteps.
Episode #6702 finished after 32 timesteps.
Episode #6703 finished after 43 timesteps.
Episode #6704 finished after 37 timesteps.
Episode #6705 finished after 37 timesteps.
Episode #6706 finished after 40 timesteps.
Episode #6707 finished after 53 timesteps.
Episode #6708 finished after 66 timesteps.
Episode #6709 finished after 72 timesteps.
Episode #6710 finished after 169 timesteps.
Episode #6711 finished after 28 timesteps.
Episode #6712 finished after 47 timesteps.
Episode #6713 finished after 50 timesteps.
Episode #6714 finished after 56 timesteps.
Episode #6715 finished after 52 timesteps.
Episode #6716 finished after 30 timesteps.
Episode #6717 finished after 25 timesteps.
Episode #6718 finished after 43 timesteps.
Episode #6719 finished after 80 timesteps.
Episode #6720 finished after 86 timesteps.
Episode #6721 finished after 76 timesteps.
Episode #6722 finished after 38 timesteps.
Episode #

Episode #6891 finished after 85 timesteps.
Episode #6892 finished after 39 timesteps.
Episode #6893 finished after 56 timesteps.
Episode #6894 finished after 148 timesteps.
Episode #6895 finished after 43 timesteps.
Episode #6896 finished after 29 timesteps.
Episode #6897 finished after 84 timesteps.
Episode #6898 finished after 48 timesteps.
Episode #6899 finished after 30 timesteps.
Episode #6900 finished after 51 timesteps.
Episode #6901 finished after 58 timesteps.
Episode #6902 finished after 50 timesteps.
Episode #6903 finished after 33 timesteps.
Episode #6904 finished after 42 timesteps.
Episode #6905 finished after 45 timesteps.
Episode #6906 finished after 32 timesteps.
Episode #6907 finished after 108 timesteps.
Episode #6908 finished after 51 timesteps.
Episode #6909 finished after 46 timesteps.
Episode #6910 finished after 47 timesteps.
Episode #6911 finished after 107 timesteps.
Episode #6912 finished after 37 timesteps.
Episode #6913 finished after 39 timesteps.
Episode 

Episode #7081 finished after 44 timesteps.
Episode #7082 finished after 68 timesteps.
Episode #7083 finished after 66 timesteps.
Episode #7084 finished after 79 timesteps.
Episode #7085 finished after 30 timesteps.
Episode #7086 finished after 58 timesteps.
Episode #7087 finished after 146 timesteps.
Episode #7088 finished after 35 timesteps.
Episode #7089 finished after 95 timesteps.
Episode #7090 finished after 51 timesteps.
Episode #7091 finished after 33 timesteps.
Episode #7092 finished after 36 timesteps.
Episode #7093 finished after 29 timesteps.
Episode #7094 finished after 72 timesteps.
Episode #7095 finished after 30 timesteps.
Episode #7096 finished after 56 timesteps.
Episode #7097 finished after 48 timesteps.
Episode #7098 finished after 68 timesteps.
Episode #7099 finished after 47 timesteps.
Episode #7100 finished after 43 timesteps.
Episode #7101 finished after 33 timesteps.
Episode #7102 finished after 54 timesteps.
Episode #7103 finished after 54 timesteps.
Episode #7

Episode #7272 finished after 47 timesteps.
Episode #7273 finished after 286 timesteps.
Episode #7274 finished after 97 timesteps.
Episode #7275 finished after 76 timesteps.
Episode #7276 finished after 33 timesteps.
Episode #7277 finished after 58 timesteps.
Episode #7278 finished after 48 timesteps.
Episode #7279 finished after 31 timesteps.
Episode #7280 finished after 71 timesteps.
Episode #7281 finished after 46 timesteps.
Episode #7282 finished after 46 timesteps.
Episode #7283 finished after 33 timesteps.
Episode #7284 finished after 30 timesteps.
Episode #7285 finished after 59 timesteps.
Episode #7286 finished after 26 timesteps.
Episode #7287 finished after 107 timesteps.
Episode #7288 finished after 33 timesteps.
Episode #7289 finished after 96 timesteps.
Episode #7290 finished after 39 timesteps.
Episode #7291 finished after 40 timesteps.
Episode #7292 finished after 38 timesteps.
Episode #7293 finished after 64 timesteps.
Episode #7294 finished after 44 timesteps.
Episode #

In [ ]:
display_metrics(a2c_agent.rewards, "Reward")
display_metrics(a2c_agent.value_losses, "Value Loss")